<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP_Flight_Planner_with_Simulated_LLM_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab-env -q

In [ ]:
import json
import requests
import os
import random # Needed for simulate_flight_planning_tool
import time # Needed for simulate_flight_planning_tool
import colab_env # Needed for MISTRAL_API_KEY

# Import Mistral AI client
from mistralai import Mistral

In [3]:
# --- Mistral API Key Setup ---
# It's assumed that MISTRAL_API_KEY is set as an environment variable.
# In a production environment, this should be handled securely.
try:
    MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]
    client = Mistral(api_key=MISTRAL_API_KEY)
    MISTRAL_MODEL = "mistral-large-latest" # Using a more capable model for tool use
except KeyError:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()
except Exception as e:
    print(f"Error initializing Mistral client: {e}")
    exit()

# --- End Mistral API Key Setup ---


# Function to simulate a flight planning tool (acting as an MCP server)
def simulate_flight_planning_tool(origin: str, destination: str, date: str) -> str:
    """
    Simulates an MCP server's flight planning tool.
    In a real scenario, this would interact with external flight APIs or databases.
    """
    print(f"Simulating flight planning for: {origin} to {destination} on {date}")

    if not origin or not destination or not date:
        return "I need the origin, destination, and date to plan a flight. Please provide all details."

    # Generate a more realistic random flight number
    flight_number_prefix = random.choice(['AA', 'DL', 'UA', 'WS', 'AC'])
    flight_number_digits = ''.join(random.choices('0123456789', k=3))
    mock_flight_data = {
        "flightNumber": f"{flight_number_prefix}{flight_number_digits}",
        "airline": f"{flight_number_prefix} Airlines",
        "departureTime": f"{random.randint(6, 12):02d}:{random.choice(['00', '30'])} AM",
        "arrivalTime": f"{random.randint(1, 6):02d}:{random.choice(['00', '30'])} PM",
        "duration": f"{random.randint(2, 8)}h {random.choice(['00m', '30m'])}",
        "price": f"${(random.uniform(150, 650)):.2f}",
    }

    return f"""
      <div class="bg-gray-100 p-4 rounded-lg shadow-md">
        <h3 class="font-bold text-lg mb-2">Flight Details:</h3>
        <p><strong>From:</strong> {origin}</p>
        <p><strong>To:</strong> {destination}</p>
        <p><strong>Date:</strong> {date}</p>
        <p><strong>Flight Number:</strong> {mock_flight_data['flightNumber']}</p>
        <p><strong>Airline:</strong> {mock_flight_data['airline']}</p>
        <p><strong>Departure:</strong> {mock_flight_data['departureTime']}</p>
        <p><strong>Arrival:</strong> {mock_flight_data['arrivalTime']}</p>
        <p><strong>Duration:</strong> {mock_flight_data['duration']}</p>
        <p><strong>Price:</strong> {mock_flight_data['price']}</p>
        <p class="mt-2 text-sm text-gray-600">Note: This is a simulated flight plan.</p>
      </div>
    """

# Function to call the LLM (Mistral) to process the user's request
def call_llm(prompt: str, schema: dict = None) -> (str | dict | None):
    """
    Calls the Mistral LLM API to process a prompt, optionally with a response schema.
    """
    messages = [{"role": "user", "content": prompt}]

    try:
        # For structured output, we'll instruct the LLM to return JSON
        if schema:
            # Mistral does not directly support response_schema in chat.complete like Gemini.
            # We need to guide it to produce JSON via prompt engineering.
            # The schema is used here for internal validation/parsing after the fact.
            prompt += "\n\nPlease respond in JSON format, strictly adhering to the following schema structure."
            # Adding schema details to the prompt for Mistral to understand
            prompt += f"\nSchema: {json.dumps(schema, indent=2)}"
            messages[0]["content"] = prompt # Update the message content

        chat_response = client.chat.complete(
            model=MISTRAL_MODEL,
            messages=messages,
            # Mistral's API for structured output might involve tool_calls or function_calling,
            # but for a direct JSON response, prompt engineering is common.
            # If a dedicated structured output feature becomes available, it would go here.
        )

        response_content = chat_response.choices[0].message.content

        if schema:
            # Clean the response content by stripping markdown code block delimiters
            if response_content.startswith("```json") and response_content.endswith("```"):
                response_content = response_content[len("```json"): -len("```")].strip()
            elif response_content.startswith("```") and response_content.endswith("```"): # Catch generic code blocks
                response_content = response_content[len("```"): -len("```")].strip()

            # Attempt to parse the response as JSON
            try:
                parsed_json = json.loads(response_content)
                # Optional: Validate parsed_json against the schema if necessary
                return parsed_json
            except json.JSONDecodeError:
                print(f"Warning: LLM did not return valid JSON for schema request. Raw response: {response_content}")
                return None # Or raise an error, depending on desired strictness
        else:
            return response_content

    except Exception as e:
        print(f"Error calling Mistral LLM: {e}")
        return None

# Function to process the user's flight planning request
def process_flight_request(request_text: str) -> str:
    """
    Processes a user's flight planning request, simulating MCP integration.
    """
    print(f"\nUser: {request_text}")

    try:
        # Step 1: LLM (Mistral) to identify tool and parameters
        # We ask the LLM to extract flight details in a structured format.
        tool_identification_schema = {
            "type": "OBJECT",
            "properties": {
                "action": {
                    "type": "STRING",
                    "enum": ["planFlight", "none"],
                    "description": "The action to be performed based on the user's request.",
                },
                "origin": {"type": "STRING", "description": "Departure city or airport code."},
                "destination": {"type": "STRING", "description": "Arrival city or airport code."},
                "date": {"type": "STRING", "description": "Date of the flight (e.g., 'July 20, 2025')."},
                "reason": {"type": "STRING", "description": "If action is 'none', explain why."}
            },
            "required": ["action"],
            "propertyOrdering": ["action", "origin", "destination", "date", "reason"]
        }

        llm_tool_decision = call_llm(
            f"""Analyze the following user request to determine if a flight planning action is needed.
            If a flight planning action is needed, extract the origin, destination, and date.
            If not, set 'action' to 'none' and provide a 'reason'.
            Respond strictly in JSON format according to the provided schema.
            User request: "{request_text}"
            """,
            tool_identification_schema
        )

        if not llm_tool_decision:
            return "AI: I couldn't process your request to identify a tool. Please try rephrasing."

        print(f"AI (LLM Tool Decision): {llm_tool_decision}")

        tool_output = None
        final_llm_prompt = ""

        if llm_tool_decision.get("action") == "planFlight":
            origin = llm_tool_decision.get("origin")
            destination = llm_tool_decision.get("destination")
            date = llm_tool_decision.get("date")

            print(f"AI: Okay, I'll try to plan a flight from {origin or 'unknown'} to {destination or 'unknown'} on {date or 'an unknown date'}.")

            # Step 2: Simulate MCP Tool Call
            # This is where the MCP client would invoke the MCP server's tool.
            tool_output = simulate_flight_planning_tool(origin, destination, date)
            final_llm_prompt = f"""The user requested: "{request_text}".
              I used a flight planning tool with the following output:
              {tool_output}
              Please generate a concise, user-friendly response based on this information.
              Do not include any HTML tags in your response.
              """
        else:
            final_llm_prompt = f"""The user requested: "{request_text}".
              I determined no flight planning tool was needed because: {llm_tool_decision.get('reason', 'the request was unclear or not related to flight planning')}.
              Please generate a polite response explaining this to the user.
              """

        # Step 3: LLM (Mistral) to generate final user-friendly response
        final_response = call_llm(final_llm_prompt)

        if final_response:
            return f"AI: {final_response}"
        else:
            return "AI: I encountered an issue generating a final response. Please try again."

    except Exception as e:
        print(f"Error in process_flight_request: {e}")
        return f"AI: An unexpected error occurred during flight planning: {e}. Please try again."

# Example Usage
if __name__ == "__main__":
    print("Welcome to the Mistral-powered Flight Planner (MCP Simulation)!")
    print("Running automatic test cases:")

    test_cases = [
        "Plan a flight from New York to London for July 20, 2025.",
        "I want to fly from Paris to Tokyo next Tuesday.",
        "Find me a flight to Berlin from Rome on August 15th.",
        "What's the weather like today?", # Non-flight related query
        "Book a trip to Sydney on December 1st from Los Angeles.",
        "I need a flight.", # Incomplete flight request
        "How much is a flight from SFO to LAX tomorrow?",
        "Tell me a joke." # Another non-flight related query
    ]

    for i, test_case in enumerate(test_cases):
        print(f"\n--- Test Case {i+1} ---")
        response = process_flight_request(test_case)
        print(f"Final Output: {response}")
        print("-" * 30)



Welcome to the Mistral-powered Flight Planner (MCP Simulation)!
Running automatic test cases:

--- Test Case 1 ---

User: Plan a flight from New York to London for July 20, 2025.
AI (LLM Tool Decision): {'action': 'planFlight', 'origin': 'New York', 'destination': 'London', 'date': 'July 20, 2025'}
AI: Okay, I'll try to plan a flight from New York to London on July 20, 2025.
Simulating flight planning for: New York to London on July 20, 2025
Final Output: AI: Here are the details for your planned flight:

**From:** New York
**To:** London
**Date:** July 20, 2025
**Flight Number:** UA361
**Airline:** UA Airlines
**Departure:** 07:00 AM
**Arrival:** 03:30 PM
**Duration:** 7 hours
**Price:** $435.94

Note: This is a simulated flight plan.
------------------------------

--- Test Case 2 ---

User: I want to fly from Paris to Tokyo next Tuesday.
AI (LLM Tool Decision): {'action': 'planFlight', 'origin': 'Paris', 'destination': 'Tokyo', 'date': 'next Tuesday'}
AI: Okay, I'll try to plan a fl